In [3]:
from astroquery.gaia import Gaia

In [4]:
query = """
SELECT SOURCE_ID, RA, DEC, PARALLAX
FROM gaiadr3.gaia_source
"""

job = Gaia.launch_job(query)

results = job.get_results()

number_of_rows = len(results)
print(f"Number of rows: {number_of_rows}")


Number of rows: 2000


In [5]:
print(f"{'Source ID':<20} {'RA (deg)':<15} {'Dec (deg)':<15} {'Parallax (mas)':<15}")

# Print a separator
print('-' * 65)

# Print each row in the results
for row in results:
    print(f"{row['SOURCE_ID']:<20} {row['RA']:<15.6f} {row['DEC']:<15.6f} {row['PARALLAX']:<15.6f}")

Source ID            RA (deg)        Dec (deg)       Parallax (mas) 
-----------------------------------------------------------------
1478150451333758336  214.358758      33.074721       -0.125318      
1478150485693118848  214.373833      33.082564       --
1478150520053236096  214.358571      33.090386       1.306953       
1478150554412488960  214.361251      33.097772       0.447386       
1478150554412489344  214.360597      33.101502       1.848563       
1478150588767848960  214.379673      33.095885       --
1478150657491750272  214.379295      33.115536       -0.522572      
1478150687552770944  214.384010      33.112236       -0.352382      
1478150829290398080  214.337431      33.114662       3.872579       
1478150893710222080  214.375611      33.138779       -0.309914      
1478150966729351680  214.362414      33.140951       3.709824       
1478151031150246400  214.418400      33.139821       0.901738       
1478151035448825472  214.416550      33.127693       0.859822  

C:\Users\sorin\AppData\Local\Temp\ipykernel_45632\4198367839.py:8: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  print(f"{row['SOURCE_ID']:<20} {row['RA']:<15.6f} {row['DEC']:<15.6f} {row['PARALLAX']:<15.6f}")


In [22]:
import numpy as np
import pandas as pd

# Extract RA and Dec

coordinates = results[['RA', 'DEC']]

parallax = np.array(results['PARALLAX'], dtype=float)
star_ra = np.array(results['RA'], dtype=float)
star_dec = np.array(results['DEC'], dtype=float)

# Handle invalid parallax values (e.g., negative or zero)
valid_parallax = np.where(parallax > 0, parallax, np.nan)  # set invalid values to NaN

# Calculate distance in parsecs (1 / parallax in arcseconds)
distance = 1 / (valid_parallax / 1000)  # convert parallax from milliarcseconds to arcseconds

# Prepare a list of coordinates as a string for Aladin Lite
coords_str = ', '.join([f"{row['RA']},{row['DEC']}" for row in coordinates])

print(distance)

# 93.1846000 d (P, Exo)

[          nan           nan  765.13855449 ...           nan 2687.1389464
           nan]


In [23]:
import math

In [57]:
def reposition(ex_ra, ex_dec, ex_dist):
    global distance, star_ra, star_dec

    new_coord = []
    ex_ra = ex_ra * math.pi/180
    ex_dec = ex_dec * math.pi/180


    for s_dist, s_ra, s_dec in zip(distance, star_ra, star_dec):
        if not math.isnan(s_dist) and not math.isnan(s_ra) and not math.isnan(s_dec):

            s_ra = s_ra * math.pi/180
            s_dec = s_dec * math.pi/180

            print("dist: ", s_dist, ex_dist)
            print("dec: ", s_dec, ex_dec)
            print("ra: ", s_ra, ex_ra)


            gamma = s_ra - ex_ra

            aux = s_dist**2 * math.cos(s_dec)**2 + ex_dist**2 * math.cos(ex_dec)**2 - 2 * s_dist * math.cos(s_dec) * ex_dist * math.cos(ex_dec) * math.cos(gamma)

            print(aux, (ex_dist**2 * math.cos(ex_dec)**2 + aux -s_dist**2 * math.cos(s_dec)**2) / (2 * ex_dist * math.cos(ex_dec) * math.sqrt(aux)))

            new_ra = math.acos((ex_dist**2 * math.cos(ex_dec)**2 + aux -s_dist**2 * math.cos(s_dec)**2) / (2 * ex_dist * math.cos(ex_dec) * aux) )


            new_dec = math.atan((s_dist * math.sin(s_dec) - ex_dist * math.sin(ex_dec)) / math.sqrt(aux))
            new_dist = (s_dist * math.sin(s_dec) - ex_dist * math.sin(ex_dec)) / math.sin(new_dec)
            new_coord.append((new_dist, new_ra, new_dec))
    
    return new_coord


In [58]:
reposition(185.1787793,17.7932516,93.1846000)

dist:  765.1385544907209 93.1846
dec:  0.5775361899130146 0.3105508250557491
ra:  3.7412628386198197 3.2319794036089204
319486.81892462773 -0.8332193548995075
dist:  2235.2062217340876 93.1846
dec:  0.5776650924919667 0.3105508250557491
ra:  3.741309617254114 3.2319794036089204
3224099.2448028647 -0.8610697609888247
dist:  540.9606979607609 93.1846
dec:  0.5777301993927507 0.3105508250557491
ra:  3.741298205113764 3.2319794036089204
143021.7252374906 -0.811568898344029
dist:  258.22585042729236 93.1846
dec:  0.577959889965739 0.3105508250557491
ra:  3.7408938839052763 3.2319794036089204
21134.711452268282 -0.6888842393279865
dist:  269.5545481492589 93.1846
dec:  0.5784187093694794 0.3105508250557491
ra:  3.7413299136599867 3.2319794036089204
23847.301630658272 -0.7014875219267857
dist:  1108.969309550872 93.1846
dec:  0.5783989867817833 0.3105508250557491
ra:  3.7423070632683215 3.2319794036089204
726353.8813039515 -0.8466158381041023
dist:  1163.030825479623 93.1846
dec:  0.578187318

[(np.float64(686.3020677692293), 1.5722704474597133, 0.6030842555877927),
 (np.float64(2155.274114153925), 1.571275877080165, 0.5860917325351536),
 (np.float64(462.91165038855206), 1.5729423021115627, 0.6146697303648093),
 (np.float64(183.88277569427194), 1.5755349231135438, 0.6590071953289056),
 (np.float64(194.89004231432756), 1.5753388988131083, 0.6561062598645804),
 (np.float64(1029.6522833353372), 1.5717896997441398, 0.5957637288767708),
 (np.float64(1083.6545388408147), 1.571741059526727, 0.5947000070477357),
 (np.float64(549.025868937211), 1.5726223171921627, 0.6099300787140569),
 (np.float64(418.90090434848565), 1.5735957722740614, -1.0794076575133955),
 (np.float64(437.77298942197615), 1.5735094064323587, -1.0735479402374875),
 (np.float64(414.888927153445), 1.573615936321875, -1.0808618280231712),
 (np.float64(6428.676375361928), 1.5709952485358394, -0.9348600953306115),
 (np.float64(2657.9035609092284), 1.5712785769510447, -0.9503879690494754),
 (np.float64(818.1177327099714

In [67]:
import csv 

data = []

with open("planet_data.csv", "r") as file:
    csv_reader = csv.reader(file)

    for row in csv_reader:
        data.append(row)

header = data[0][1:]
data = data[1:]

exo_dict = {}

for row in data:
    exo_dict[row[0]] = {}

    for name, info in zip(header, row[1:]):
        exo_dict[row[0]][name] = info 

print(exo_dict)

{'11 Com b': {'hostname': '11 Com', 'ra': '185.1787793', 'dec': '17.7932516', 'sy_dist': '93.1846000', 'sy_plx': '10.7104000', 'sy_gaiamag': '4.4403800'}, '11 UMi b': {'hostname': '11 UMi', 'ra': '229.2745954', 'dec': '71.8239428', 'sy_dist': '125.3210000', 'sy_plx': '7.9538800', 'sy_gaiamag': '4.5621600'}, '14 And b': {'hostname': '14 And', 'ra': '352.8241500', 'dec': '39.2358367', 'sy_dist': '75.4392000', 'sy_plx': '13.2289000', 'sy_gaiamag': '4.9178100'}, '14 Her b': {'hostname': '14 Her', 'ra': '242.6021014', 'dec': '43.8163621', 'sy_dist': '17.9323000', 'sy_plx': '55.7363000', 'sy_gaiamag': '6.3830000'}, '16 Cyg B b': {'hostname': '16 Cyg B', 'ra': '295.4656422', 'dec': '50.5168243', 'sy_dist': '21.1397000', 'sy_plx': '47.2754000', 'sy_gaiamag': '6.0642800'}, '17 Sco b': {'hostname': '17 Sco', 'ra': '243.4620874', 'dec': '-11.8377905', 'sy_dist': '124.9530000', 'sy_plx': '7.9805000', 'sy_gaiamag': '4.7542900'}, '18 Del b': {'hostname': '18 Del', 'ra': '314.6078375', 'dec': '10.839

In [35]:
import csv
import math

data={"RA": [], "DEC": [], "PRLX":[], "MAG": []}
with open("data12.csv", mode='r', newline='') as file:
    out = open("data122.csv", mode='w', newline='')
    reader = csv.reader(file)
    writer = csv.writer(out)
    next(reader)
    writer.writerow(["sourceid","RA","Dec","prlx","magnitude"])
    
    for row in reader:
        data["RA"].append(float(row[1]))
        data["DEC"].append(float(row[2]))
        data["PRLX"].append(float(row[3]))
        distance = 1 / (float(data["PRLX"][-1]) / 1000)
        data["MAG"].append(float(row[4]) + 5 - 5 * math.log10(distance))
        print(row[0])
        writer.writerow([row[0], data["RA"][-1], data["DEC"][-1], data["PRLX"][-1], data["MAG"][-1]])
        


418551920284673408
4357027756659697664
5589311357728452608
4993479684438433792
4038055447778237312
1279798794197267072
160886283751041408
4302054339959905920
1222646935698492160
5111187420714898304
1563590579347125632
4049506483413484672
6227443304915069056
804753180515722624
4429785739602747392
2106630885454013184
5922444483103229952
6407842789021567872
856096765753549056
3704342295607157120
4473334474604992384
3501215734352781440
2202630001603369856
4629125170492116224
702343774145932544
1625209684868707328
6126469654585981952
5917537534527580160
5361403934691772160
4076915349846977664
5306951892646516992
1297565458994346240
1415230383034813824
2971266110612490624
4337352305320149760
5099455184968674560
5502164890740285696
3736865265441207424
2968097043228517120
3490234568129160704
5512070906394195968
3384529341302871296
447071293401293056
2026116260337482112
2900546759663847168
2962546605447869184
779106556394253824
510204838759030144
4683897617110115200
1339952869195300608
58594058